# 웹크롤링

## 네이버 메인 뉴스목록 추출

In [1]:
import requests
url = 'https://www.naver.com/'
res = requests.get(url).text
res[:200]

'\n<!doctype html>                          <html lang="ko" data-dark="false"> <head> <meta charset="utf-8"> <title>NAVER</title> <meta http-equiv="X-UA-Compatible" content="IE=edge"> <meta name="viewpo'

In [2]:
# 뉴스목록 파싱
from bs4 import BeautifulSoup as bs
import urllib.request as req
# response = req.urlopen(url)
soup = bs(res,"html.parser")
# print(soup.head)   #검색 결과 태그
print(type(soup.head))  # 검색결과 태그
print(soup.title.string)  #해당 태그가 직접 가지고 있는 글자
print(soup.title.get_text())   #해당 태그 내 모든 태그들이 가지고 있는 글자

<class 'bs4.element.Tag'>
NAVER
NAVER


In [3]:
news_list = soup.select('div.list_issue > div')
len(news_list)

10

In [4]:
news_title = []
for news in news_list:
    news_title.append(news.get_text())
news_title

['여야, 일시 2주택·고령자-장기보유 등 종부세 부담 완화 합의',
 '원/달러 환율, 연고점 경신…장 중 1,353원까지 올라',
 "'尹心' 작용? 윤핵관 2선 후퇴속 與 '추석전' 새 비대위 속도전",
 "검찰, '서해 공무원 피격' 文정부 대통령기록물 압수수색",
 '한동훈, 정경심 형집행정지 불허에 "관여 안했지만 살펴볼것"',
 '이재명 "내복 사입은 것밖에는"…\'쌍방울 커넥션\' 與 주장 반박',
 "검찰, '尹 찍어내기 감찰 의혹' 박은정 휴대전화 압수수색",
 '국민연금공단 이사장에 김태현 내정…내일 취임식',
 '8월 수입액 역대 1위에 무역적자 100억달러 육박…66년만에 최대',
 "전세계약 직후 집주인 대출·매매 금지…'특약'으로 규제한다"]

In [6]:
# 네이버 환율검색 결과 화면에서 통화명, 매매기준율 추출하여 사전객체에 저장
# {'미국 USD':1,350.50, '일본 JPY':967.37, ...}
import requests
from bs4 import BeautifulSoup as bs
import urllib.request as req

url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%ED%99%98%EC%9C%A8'
res = requests.get(url).text
soup = bs(res,"html.parser")
table = soup.select('table.rate_table_info > tbody > tr')

table_list = []
table_list1 = []

for line in table:
    table_list.append(line.th.get_text())
    table_list1.append(line.td.get_text())

table_dict = dict(zip(table_list, table_list1))
table_dict

{'미국 USD': '1,353.90',
 '일본 JPY 100': '970.57',
 '유럽연합 EUR': '1,356.34',
 '중국 CNY': '195.71',
 '영국 GBP': '1,567.88',
 '호주 AUD': '921.67',
 '캐나다 CAD': '1,028.53',
 '뉴질랜드 NZD': '824.59'}

In [90]:
# 네이버 환율검색 결과 화면에서 통화명, 매매기준율 추출하여 사전객체에 저장
# {'미국 USD':1,350.50, '일본 JPY':967.37, ...}
import requests
from bs4 import BeautifulSoup as bs
import urllib.request as req

url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%ED%99%98%EC%9C%A8'
res = requests.get(url).text
soup = bs(res,"html.parser")
table = soup.select('table.rate_table_info > tbody > tr')
table[1].select('td')

[<td><span>972.47</span></td>,
 <td class="flu_nm"><span class="blind">전일대비</span><span class="ico">상승</span><em>3.29</em></td>,
 <td class="flu_pct"><span>+0.34%</span></td>]

</br>
</br>

## KBO 일정 Ajax 추출

In [80]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.request as req

url_kbo = 'https://www.koreabaseball.com/ws/Schedule.asmx/GetScheduleList'
params = {'leId':"1",'srIdList':"0,9",'seasonId':'2022','gameMonth':"07",'teamId':""}
res_kbo = requests.post(url_kbo, params).json()
res_kbo

{'colgroup': [],
 'headers': [],
 'rows': [{'row': [{'Text': '07.01(금)',
     'Class': 'day',
     'Scope': None,
     'RowSpan': '5',
     'ColSpan': None,
     'Width': None,
     'TypeObj': None},
    {'Text': '<b>18:30</b>',
     'Class': 'time',
     'Scope': None,
     'RowSpan': None,
     'ColSpan': None,
     'Width': None,
     'TypeObj': None},
    {'Text': '<span>롯데</span><em><span class="lose">1</span><span>vs</span><span class="win">2</span></em><span>LG</span>',
     'Class': 'play',
     'Scope': None,
     'RowSpan': None,
     'ColSpan': None,
     'Width': None,
     'TypeObj': None},
    {'Text': "<a href='/Schedule/GameCenter/Main.aspx?gameDate=20220701&gameId=20220701LTLG0&section=REVIEW' class='btn2 mr5' id='btnReview'>리뷰</a>",
     'Class': 'relay',
     'Scope': None,
     'RowSpan': None,
     'ColSpan': None,
     'Width': None,
     'TypeObj': None},
    {'Text': "<a href='/Schedule/GameCenter/Main.aspx?gameDate=20220701&gameId=20220701LTLG0&section=HIGHLIGH

In [40]:
import re

def Name(s=''):
    name = re.sub(r'[0-9]+','',s.replace('vs',':'))
    return name
    
def Num(s=''):
    numbers = re.sub(r'[a-zA-Z]+','',re.sub(r'[ㄱ-ㅣ가-힣]+','',s.replace('vs',':')))
    return numbers

keys_list = ['일시','시간','팀','결과','구장']
val_list = []
list_fin = []

for i in range(len(res_kbo['rows'])):
    tmp_list = []
    tmp1 = []
    for j in range(len(res_kbo['rows'][i]['row'])):
        if res_kbo['rows'][i]['row'][j]['Class'] == 'day':
            date = str(res_kbo['rows'][i]['row'][j]['Text'])
        else:
            tmp_str = re.sub('<.+?>', "", str(res_kbo['rows'][i]['row'][j]['Text']))
            if tmp_str != '':
                tmp1.append(tmp_str)
            if 'vs' in tmp_str:
                team = Name(str(tmp_str))
                score = Num(str(tmp_str))
    if tmp1[-1] != "-":
        score = '경기취소'
    tmp_list.append(date)
    tmp_list.append(tmp1[0])
    tmp_list.append(team)
    tmp_list.append(score)
    tmp_list.append(tmp1[-2])
    val_list.append(tmp_list)
for i in range(len(val_list)):
    list_fin.append(dict(zip(keys_list,val_list[i])))

with open('크보일정결과.txt','w') as f:
    for line in list_fin:
        f.write(str(line).strip('{,}')+'\n')
f.close()

with open('크보일정결과.csv','w') as f:
    f.write(str(keys_list).strip('[,]') + '\n')
    for line in val_list:
        f.write(str(line).strip('[,]') + '\n')
f.close()

</br>
</br>

## 이미지 다운로드 크롤링

</br>
</br>

In [79]:
#google에서 이미지 크롤링 하기
## 검색어를 지정하여 크롤링하기

import requests
from bs4 import BeautifulSoup as bs
import urllib.request as req

def get_keyword(s='악어'):
    keyword = s
    return keyword

url_search = 'https://www.google.com/search?q='
keyword = get_keyword()
params = ['&sxsrf=ALiCzsZlfR9rfXhyqSk2yoEWVdzrEK8yew:1662028413834&source=lnms&tbm=isch&sa=X&ved=2ahUKEwi199mpsvP5AhUW_GEKHQBPDTMQ_AUoAXoECAIQAw']
res_img = requests.get(url_search+keyword+str(params)).text
soup = bs(res_img,"html.parser")
tmp = soup.find('body',{'id':'yDmH0d'})
print(tmp)

None


In [ ]:
def get_link():
    pass
